In [1]:
# Import the necessary libraries
from PIL import Image
from numpy import asarray
import numpy as np
import os
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os
from keras.utils import np_utils
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
# calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
from sklearn.metrics import f1_score
print('all imported')

all imported


In [2]:
def readImage(filePath):
    img = Image.open(filePath)
    
    size=(30,40)
    #resize image
    out = img.resize(size)

    # asarray() class is used to convert
    # PIL images into NumPy arrays
    numpydata = asarray(out)
    numpydata = np.repeat(numpydata[:, :, np.newaxis], 3, axis=2)
    # <class 'numpy.ndarray'>
    #print(type(numpydata))

    #  shape
    #print(numpydata.shape)
    return numpydata
print(readImage('data/disease/measles (1).png'))

[[[[ 76  84  95]
   [ 76  84  95]
   [ 76  84  95]]

  [[ 15  15  20]
   [ 15  15  20]
   [ 15  15  20]]

  [[  8   7  11]
   [  8   7  11]
   [  8   7  11]]

  ...

  [[ 16  14  25]
   [ 16  14  25]
   [ 16  14  25]]

  [[ 13  13  22]
   [ 13  13  22]
   [ 13  13  22]]

  [[  8   9  13]
   [  8   9  13]
   [  8   9  13]]]


 [[[ 88  96 107]
   [ 88  96 107]
   [ 88  96 107]]

  [[ 17  17  22]
   [ 17  17  22]
   [ 17  17  22]]

  [[  6   5   9]
   [  6   5   9]
   [  6   5   9]]

  ...

  [[ 15  13  23]
   [ 15  13  23]
   [ 15  13  23]]

  [[ 15  13  24]
   [ 15  13  24]
   [ 15  13  24]]

  [[ 10  10  16]
   [ 10  10  16]
   [ 10  10  16]]]


 [[[ 86  91  98]
   [ 86  91  98]
   [ 86  91  98]]

  [[ 14  13  17]
   [ 14  13  17]
   [ 14  13  17]]

  [[  5   4   7]
   [  5   4   7]
   [  5   4   7]]

  ...

  [[ 15  13  23]
   [ 15  13  23]
   [ 15  13  23]]

  [[ 15  13  23]
   [ 15  13  23]
   [ 15  13  23]]

  [[ 12  11  18]
   [ 12  11  18]
   [ 12  11  18]]]


 ...


 [[[157 162 

In [3]:
listFiles = os.listdir('data/disease/')
print(listFiles[:5])

['4164443_down-syndrome-down-syndrome-transparent-background-transparent-png.png', 'main-qimg-4c812ebe5c1bdc504625b6961032fd3b.png', 'measles (1).png', 'measles (10).jpg', 'measles (140).jpg']


In [4]:
#load Benign
listCancer = []
listResults = []
for elem in listFiles:
    img = readImage('data/disease/'+elem)
    listCancer += [img]
    listResults += [np.array([1])]
print(listCancer[:3])
print(img.shape)

[array([[[[243, 243, 243],
         [243, 243, 243],
         [243, 243, 243]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        ...,

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]]],


       [[[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        ...,

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]]],


       

In [5]:
listFiles = os.listdir('data/health/')
print(listFiles[:5])
for elem in listFiles:
    img = readImage('data/health/'+elem)
    listResults += [np.array([0])]
    listCancer += [img]

['1 (1).jpeg', '1 (1).jpg', '1 (1).png', '1 (10).jpeg', '1 (10).jpg']


In [6]:
print(len(listCancer))

200


In [7]:
# Setting LR for different number of Epochs
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    lr *= epoch / 100
    print('Learning rate: ', lr)
    return lr


In [8]:
# Basic ResNet Building Block
def resnet_layer(inputs, conv_first = False,
				num_filters = 16,
				kernel_size = 3,
				strides = 1,
				activation ='relu',
				batch_normalization = True):
	conv = Conv2D(num_filters,
				kernel_size = kernel_size,
				strides = strides,
				padding ='same',
				kernel_initializer ='he_normal',
				kernel_regularizer = l2(1e-4))

	x = inputs
	if conv_first:
		x = conv(x)
		if batch_normalization:
			x = BatchNormalization()(x)
		if activation is not None:
			x = Activation(activation)(x)
	else:
		if batch_normalization:
			x = BatchNormalization()(x)
		if activation is not None:
			x = Activation(activation)(x)
		x = conv(x)
	return x

In [9]:
# ResNet V2 architecture
def resnet_v2(input_shape, depth, num_classes = 10):
	if (depth - 2) % 9 != 0:
		raise ValueError('depth should be 9n + 2 (eg 56 or 110 in [b])')
	# Start model definition.
	num_filters_in = 16
	num_res_blocks = int((depth - 2) / 9)

	inputs = Input(shape = input_shape)
	# v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
	x = resnet_layer(inputs = inputs,num_filters = num_filters_in,conv_first = True)

	# Instantiate the stack of residual units
	for stage in range(3):
		for res_block in range(num_res_blocks):
			activation = 'relu'
			batch_normalization = True
			strides = 1
			if stage == 0:
				num_filters_out = num_filters_in * 4
				if res_block == 0: # first layer and first stage
					activation = None
					batch_normalization = False
			else:
				num_filters_out = num_filters_in * 2
				if res_block == 0: # first layer but not first stage
					strides = 2 # downsample

			# bottleneck residual unit
			y = resnet_layer(inputs = x,
							num_filters = num_filters_in,
							kernel_size = 1,
							strides = strides,
							activation = activation,
							batch_normalization = batch_normalization)
			y = resnet_layer(inputs = y,
							num_filters = num_filters_in,
							conv_first = False)
			y = resnet_layer(inputs = y,
							num_filters = num_filters_out,
							kernel_size = 1,
							conv_first = False)
			if res_block == 0:
				# linear projection residual shortcut connection to match
				# changed dims
				x = resnet_layer(inputs = x,
								num_filters = num_filters_out,
								kernel_size = 1,
								strides = strides,
								activation = None,
								batch_normalization = False)
			x = keras.layers.add([x, y])

		num_filters_in = num_filters_out

	# Add classifier on top.
	# v2 has BN-ReLU before Pooling
	x = BatchNormalization()(x)
	x = Activation('relu')(x)
	x = AveragePooling2D(pool_size = 8)(x)
	y = Flatten()(x)
	outputs = Dense(num_classes,
					activation ='softmax',
					kernel_initializer ='he_normal')(y)

	# Instantiate model.
	model = Model(inputs = inputs, outputs = outputs)
	return model

In [14]:
x_train=[]
y_train=[]
accs=[]
def main(pz, created_model):
    
    per = np.random.permutation(len(listCancer))
    ln = int(len(listCancer) * 0.55)
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    positions = []
    #for i in range(ln):
    #    positions += [per[i]]
    val = 1
    while ln > 0:
        val = 1-val
        ln-=1
        for i in range(len(per)):
            if per[i] in positions:
                continue
            if listResults[per[i]] == np.array([val]):
                positions += [per[i]]
                break
    for i in range(len(listCancer)):
        if len(listCancer[i].shape)==3:
            continue
        if listCancer[i].shape[3] == 4:
            continue
        #print(listCancer[i].shape)
        if i in positions:
            x_train += [listCancer[i]]
            y_train += [listResults[i]]
        else:
            x_test += [listCancer[i]]
            y_test += [listResults[i]]
            
    ln = len(x_test) // 2
    
    x_valid = x_test[ln:]
    y_valid = y_test[ln:]
    
    print(len(x_valid))
    print(len(y_valid))
    
    
    x_test = x_test[:ln]
    y_test = y_test[:ln]
    
    
    x_train = np.array(x_train)
    x_train = x_train.reshape(len(x_train), 40, 30, 9)
    y_train = np.array(y_train)
    

    x_test = np.array(x_test)
    x_test = x_test.reshape(len(x_test), 40, 30, 9)
    y_test = np.array(y_test)
    
    x_valid = np.array(x_valid)
    x_valid = x_valid.reshape(len(x_valid), 40, 30, 9)
    y_valid = np.array(y_valid)
    
    
    batch_size = 64 # original ResNet paper uses batch_size = 128 for training
    epochs = 10
    data_augmentation = True
    num_classes = 10

    # Data Preprocessing
    subtract_pixel_mean = True
    n = 3

    # Select ResNet Version
    version = 2

    # Computed depth of
    if version == 1:
        depth = n * 6 + 2
    elif version == 2:
        depth = n * 9 + 2

    # Model name, depth and version
    model_type = 'ResNet % dv % d' % (depth, version)

    # use the data
    print(x_train[:3])
    print(y_train[:3])
    print(type(x_train[0]))
    print(type(x_train))
    print(type(y_train[0]))
    print(type(y_train))

    # Input image dimensions.
    input_shape = x_train.shape[1:]
    print(input_shape)

    # Normalize data.
    x_train = x_train.astype('float32') / 255
    x_test = x_test.astype('float32') / 255
    x_valid = x_test.astype('float32') / 255


    # Print Training and Test Samples
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print('y_train shape:', y_train.shape)

    # Convert class vectors to binary class matrices.
    y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)
    y_valid = keras.utils.np_utils.to_categorical(y_valid, num_classes)
    
    
    model = resnet_v2(input_shape = input_shape, depth = depth)

    model.compile(loss ='categorical_crossentropy',
                optimizer = Adam(learning_rate = lr_schedule(0)),
                metrics =['accuracy'])
    model.summary()
    print(model_type)

    # Prepare model model saving directory.
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    model_name = 'cifar10_% s_model.{epoch:03d}.h5' % model_type
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    filepath = os.path.join(save_dir, model_name)

    # Prepare callbacks for model saving and for learning rate adjustment.
    checkpoint = ModelCheckpoint(filepath = filepath,
                                monitor ='val_acc',
                                verbose = 1,
                                save_best_only = True)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    lr_reducer = ReduceLROnPlateau(factor = np.sqrt(0.1),
                                cooldown = 0,
                                patience = 5,
                                min_lr = 0.5e-6)

    callbacks = [checkpoint, lr_reducer, lr_scheduler]
    
    while len(y_valid) > len(x_valid):
        y_valid = np.delete(y_valid, len(y_valid)-1)

    # Run training, with or without data augmentation.
    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(x_train, y_train,
                batch_size = batch_size,
                epochs = epochs,
                validation_data =(x_test, y_test),
                shuffle = True,
                callbacks = callbacks)
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            # set input mean to 0 over the dataset
            featurewise_center = False,
            # set each sample mean to 0
            samplewise_center = False,
            # divide inputs by std of dataset
            featurewise_std_normalization = False,
            # divide each input by its std
            samplewise_std_normalization = False,
            # apply ZCA whitening
            zca_whitening = False,
            # epsilon for ZCA whitening
            zca_epsilon = 1e-06,
            # randomly rotate images in the range (deg 0 to 180)
            rotation_range = 0,
            # randomly shift images horizontally
            width_shift_range = 0.1,
            # randomly shift images vertically
            height_shift_range = 0.1,
            # set range for random shear
            shear_range = 0.,
            # set range for random zoom
            zoom_range = 0.,
            # set range for random channel shifts
            channel_shift_range = 0.,
            # set mode for filling points outside the input boundaries
            fill_mode ='nearest',
            # value used for fill_mode = "constant"
            cval = 0.,
            # randomly flip images
            horizontal_flip = True,
            # randomly flip images
            vertical_flip = False,
            # set rescaling factor (applied before any other transformation)
            rescale = None,
            # set function that will be applied on each input
            preprocessing_function = None,
            # image data format, either "channels_first" or "channels_last"
            data_format = None,
            # fraction of images reserved for validation (strictly between 0 and 1)
            validation_split = 0.1)

        # Compute quantities required for featurewise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        print('------------------')
        print(x_train)
        datagen.fit(x_train)

        # Fit the model on the batches generated by datagen.flow().
        model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size),
                            validation_data =(x_valid, y_valid),
                            epochs = 100, verbose = 1, workers = 2,
                            callbacks = callbacks)

    # Score trained model.
    scores = model.evaluate(x_test, y_test, verbose = 1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    ret = model.predict(x_test)
    result = []
    pred = []
    for a,b in zip(ret, y_test):
        pred+=[a.argmax(axis=-1)]
        result+=[b.argmax(axis=-1)]
    print(pred)
    print(result)
    precision = precision_score(result, pred, average='weighted')
    recall = recall_score(result, pred, average='weighted')
     # calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
    fmeasure = f1_score(result, pred, average='weighted')
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('F1 Score: ', fmeasure)
    #precisions += [precision]
    #recalls += [recall]
    #model.save('saved_models/resNetV'+pz+'.h5')
    return [scores[1], precision, recall, fmeasure]

In [15]:
accs=[]
precisions = []
recalls = []
fmeasures = []
created_model = None
cnt = 0
while cnt < 10:
    vf=True
    #print(cnt)
    try:
        ret = main(cnt, created_model)
        cnt += 1
    except:
        vf = False
    if not vf:
        continue
    
    accs+=[ret[0]]
    precisions+=[ret[1]]
    recalls+=[ret[2]]
    fmeasures+=[ret[3]]
print(accs)
print(precisions)
print(recalls)
print(fmeasures)

29
29
[[[[ 76  84  95 ...  76  84  95]
   [ 15  15  20 ...  15  15  20]
   [  8   7  11 ...   8   7  11]
   ...
   [ 16  14  25 ...  16  14  25]
   [ 13  13  22 ...  13  13  22]
   [  8   9  13 ...   8   9  13]]

  [[ 88  96 107 ...  88  96 107]
   [ 17  17  22 ...  17  17  22]
   [  6   5   9 ...   6   5   9]
   ...
   [ 15  13  23 ...  15  13  23]
   [ 15  13  24 ...  15  13  24]
   [ 10  10  16 ...  10  10  16]]

  [[ 86  91  98 ...  86  91  98]
   [ 14  13  17 ...  14  13  17]
   [  5   4   7 ...   5   4   7]
   ...
   [ 15  13  23 ...  15  13  23]
   [ 15  13  23 ...  15  13  23]
   [ 12  11  18 ...  12  11  18]]

  ...

  [[157 162 174 ... 157 162 174]
   [159 165 176 ... 159 165 176]
   [155 158 159 ... 155 158 159]
   ...
   [126 136 168 ... 126 136 168]
   [145 153 178 ... 145 153 178]
   [150 157 180 ... 150 157 180]]

  [[158 162 174 ... 158 162 174]
   [158 164 177 ... 158 164 177]
   [160 167 179 ... 160 167 179]
   ...
   [134 144 172 ... 134 144 172]
   [143 151 177 ... 

c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\keras_preprocessing\image\image_data_generator.py:938: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (72, 40, 30, 9) (9 channels).
  warnings.warn(
c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\keras_preprocessing\image\numpy_array_iterator.py:129: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (72, 40, 30, 9) (9 channels).
  warnings.warn('NumpyArrayIterator is set to use the '
c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and wil

2/2 [==============================] - 8s 2s/step - loss: 2.3974 - accuracy: 0.5278 - val_loss: 2.8400 - val_accuracy: 1.0000
Epoch 2/100
Learning rate:  1e-05
2/2 [==============================] - 2s 2s/step - loss: 2.3965 - accuracy: 0.4722 - val_loss: 2.7963 - val_accuracy: 0.9655
Epoch 3/100
Learning rate:  2e-05
2/2 [==============================] - 2s 443ms/step - loss: 2.3861 - accuracy: 0.5000 - val_loss: 2.7647 - val_accuracy: 0.0345
Epoch 4/100
Learning rate:  3e-05
2/2 [==============================] - 2s 2s/step - loss: 2.3708 - accuracy: 0.5278 - val_loss: 2.7370 - val_accuracy: 0.0000e+00
Epoch 5/100
Learning rate:  4e-05
2/2 [==============================] - 2s 458ms/step - loss: 2.3567 - accuracy: 0.5000 - val_loss: 2.7116 - val_accuracy: 0.0000e+00
Epoch 6/100
Learning rate:  5e-05
2/2 [==============================] - 2s 2s/step - loss: 2.2951 - accuracy: 0.6111 - val_loss: 2.6808 - val_accuracy: 0.0000e+00
Epoch 7/100
Learning rate:  6e-05
2/2 [=================

2/2 [==============================] - 2s 2s/step - loss: 1.0241 - accuracy: 0.9028 - val_loss: 1.1495 - val_accuracy: 1.0000
Epoch 35/100
Learning rate:  0.00034
2/2 [==============================] - 3s 615ms/step - loss: 1.0402 - accuracy: 0.8611 - val_loss: 1.1374 - val_accuracy: 1.0000
Epoch 36/100
Learning rate:  0.00035
2/2 [==============================] - 3s 2s/step - loss: 0.9985 - accuracy: 0.9028 - val_loss: 1.1333 - val_accuracy: 1.0000
Epoch 37/100
Learning rate:  0.00035999999999999997
2/2 [==============================] - 2s 2s/step - loss: 0.9860 - accuracy: 0.8889 - val_loss: 1.0902 - val_accuracy: 1.0000
Epoch 38/100
Learning rate:  0.00037
2/2 [==============================] - 2s 441ms/step - loss: 0.9401 - accuracy: 0.8750 - val_loss: 1.0147 - val_accuracy: 1.0000
Epoch 39/100
Learning rate:  0.00038
2/2 [==============================] - 2s 467ms/step - loss: 0.9752 - accuracy: 0.8750 - val_loss: 0.9139 - val_accuracy: 1.0000
Epoch 40/100
Learning rate:  0.0003

Epoch 68/100
Learning rate:  0.00067
2/2 [==============================] - 2s 459ms/step - loss: 0.7853 - accuracy: 0.9444 - val_loss: 0.6503 - val_accuracy: 1.0000
Epoch 69/100
Learning rate:  0.00068
2/2 [==============================] - 2s 2s/step - loss: 0.7230 - accuracy: 0.9861 - val_loss: 0.9021 - val_accuracy: 1.0000
Epoch 70/100
Learning rate:  0.00069
2/2 [==============================] - 2s 489ms/step - loss: 0.6889 - accuracy: 0.9722 - val_loss: 1.1876 - val_accuracy: 1.0000
Epoch 71/100
Learning rate:  0.0007
2/2 [==============================] - 2s 461ms/step - loss: 0.6849 - accuracy: 0.9722 - val_loss: 1.3660 - val_accuracy: 1.0000
Epoch 72/100
Learning rate:  0.00071
2/2 [==============================] - 2s 454ms/step - loss: 0.6885 - accuracy: 0.9722 - val_loss: 1.6135 - val_accuracy: 1.0000
Epoch 73/100
Learning rate:  0.0007199999999999999
2/2 [==============================] - 2s 495ms/step - loss: 0.6893 - accuracy: 1.0000 - val_loss: 1.9467 - val_accuracy: 0

...

c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\keras_preprocessing\image\image_data_generator.py:938: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (70, 40, 30, 9) (9 channels).
  warnings.warn(
c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\keras_preprocessing\image\numpy_array_iterator.py:129: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (70, 40, 30, 9) (9 channels).
  warnings.warn('NumpyArrayIterator is set to use the '
c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and wil

2/2 [==============================] - 6s 2s/step - loss: 3.3289 - accuracy: 0.0286 - val_loss: 2.9949 - val_accuracy: 0.0000e+00
Epoch 2/100
Learning rate:  1e-05
2/2 [==============================] - 2s 2s/step - loss: 3.3343 - accuracy: 0.0286 - val_loss: 3.0326 - val_accuracy: 0.0000e+00
Epoch 3/100
Learning rate:  2e-05
2/2 [==============================] - 2s 315ms/step - loss: 3.3455 - accuracy: 0.0286 - val_loss: 3.0510 - val_accuracy: 0.0000e+00
Epoch 4/100
Learning rate:  3e-05
2/2 [==============================] - 2s 307ms/step - loss: 3.3208 - accuracy: 0.0286 - val_loss: 3.0548 - val_accuracy: 0.0000e+00
Epoch 5/100
Learning rate:  4e-05
2/2 [==============================] - 2s 309ms/step - loss: 3.2920 - accuracy: 0.0286 - val_loss: 3.0508 - val_accuracy: 0.0000e+00
Epoch 6/100
Learning rate:  5e-05
2/2 [==============================] - 2s 297ms/step - loss: 3.2826 - accuracy: 0.0286 - val_loss: 3.0416 - val_accuracy: 0.0000e+00
Epoch 7/100
Learning rate:  6e-05
2/2 

2/2 [==============================] - 2s 343ms/step - loss: 1.2386 - accuracy: 0.8000 - val_loss: 2.8196 - val_accuracy: 0.0000e+00
Epoch 35/100
Learning rate:  0.00034
2/2 [==============================] - 2s 304ms/step - loss: 1.2413 - accuracy: 0.8000 - val_loss: 2.8841 - val_accuracy: 0.0000e+00
Epoch 36/100
Learning rate:  0.00035
2/2 [==============================] - 2s 1s/step - loss: 1.1737 - accuracy: 0.8143 - val_loss: 2.9492 - val_accuracy: 0.0000e+00
Epoch 37/100
Learning rate:  0.00035999999999999997
2/2 [==============================] - 2s 1s/step - loss: 1.1527 - accuracy: 0.8143 - val_loss: 3.0072 - val_accuracy: 0.0000e+00
Epoch 38/100
Learning rate:  0.00037
2/2 [==============================] - 2s 321ms/step - loss: 1.1172 - accuracy: 0.8571 - val_loss: 3.0961 - val_accuracy: 0.0000e+00
Epoch 39/100
Learning rate:  0.00038
2/2 [==============================] - 2s 1s/step - loss: 1.1470 - accuracy: 0.8000 - val_loss: 3.1694 - val_accuracy: 0.0000e+00
Epoch 40/10

2/2 [==============================] - 2s 1s/step - loss: 0.7563 - accuracy: 0.9571 - val_loss: 0.5964 - val_accuracy: 1.0000
Epoch 68/100
Learning rate:  0.00067
2/2 [==============================] - 2s 313ms/step - loss: 0.7437 - accuracy: 0.9571 - val_loss: 0.5960 - val_accuracy: 1.0000
Epoch 69/100
Learning rate:  0.00068
2/2 [==============================] - 2s 1s/step - loss: 0.7648 - accuracy: 0.9571 - val_loss: 0.5956 - val_accuracy: 1.0000
Epoch 70/100
Learning rate:  0.00069
2/2 [==============================] - 1s 1s/step - loss: 0.7236 - accuracy: 0.9714 - val_loss: 0.5951 - val_accuracy: 1.0000
Epoch 71/100
Learning rate:  0.0007
2/2 [==============================] - 2s 328ms/step - loss: 0.7001 - accuracy: 1.0000 - val_loss: 0.5947 - val_accuracy: 1.0000
Epoch 72/100
Learning rate:  0.00071
2/2 [==============================] - 2s 321ms/step - loss: 0.7297 - accuracy: 0.9714 - val_loss: 0.5942 - val_accuracy: 1.0000
Epoch 73/100
Learning rate:  0.0007199999999999999


1/1 [==============================] - 0s 161ms/step - loss: 1.6228 - accuracy: 0.6000
Test loss: 1.6228071451187134
Test accuracy: 0.6000000238418579
[0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
Precision:  0.84
Recall:  0.6
F1 Score:  0.6107142857142858
[0.7241379022598267, 0.8333333134651184, 0.699999988079071, 0.8333333134651184, 0.800000011920929, 0.75, 0.931034505367279, 0.9230769276618958, 0.7096773982048035, 0.6000000238418579]
[0.871264367816092, 0.8888888888888888, 0.7625, 0.8928571428571428, 0.8, 0.8472222222222222, 0.9373040752351096, 0.9311740890688258, 0.815994623655914, 0.84]
[0.7241379310344828, 0.8333333333333334, 0.7, 0.8333333333333334, 0.8, 0.75, 0.9310344827586207, 0.9230769230769231, 0.7096774193548387, 0.6]
[0.7436433298502263, 0.8380952380952381, 0.7123384253819036, 0.8401880141010576, 0.8, 0.747765006385696, 0.9283661740558292

In [18]:
print(len(accs))
print('accs: ', accs)
print('precisions: ', precisions)
print('recalls: ', recalls)
print('fmeasures: ', fmeasures)

10
accs:  [0.7241379022598267, 0.8333333134651184, 0.699999988079071, 0.8333333134651184, 0.800000011920929, 0.75, 0.931034505367279, 0.9230769276618958, 0.7096773982048035, 0.6000000238418579]
precisions:  [0.871264367816092, 0.8888888888888888, 0.7625, 0.8928571428571428, 0.8, 0.8472222222222222, 0.9373040752351096, 0.9311740890688258, 0.815994623655914, 0.84]
recalls:  [0.7241379310344828, 0.8333333333333334, 0.7, 0.8333333333333334, 0.8, 0.75, 0.9310344827586207, 0.9230769230769231, 0.7096774193548387, 0.6]
fmeasures:  [0.7436433298502263, 0.8380952380952381, 0.7123384253819036, 0.8401880141010576, 0.8, 0.747765006385696, 0.9283661740558292, 0.9204059829059829, 0.7278023519257741, 0.6107142857142858]


In [19]:
import numpy as np
import scipy.stats as st

#define sample data
data = accs

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.7061421141092344, 0.8547765627439455)

In [1]:
accs=  [0.6153846383094788, 0.8571428656578064, 0.6428571343421936, 0.692307710647583, 0.6666666865348816, 0.6666666865348816, 0.5, 1.0, 0.7142857313156128, 0.6153846383094788]
precisions=  [0.8557692307692307, 0.9142857142857144, 0.8660714285714286, 1.0, 0.875, 0.8148148148148149, 0.888888888888889, 1.0, 0.8571428571428571, 0.8901098901098901]
recalls=  [0.6153846153846154, 0.8571428571428571, 0.6428571428571429, 0.6923076923076923, 0.6666666666666666, 0.6666666666666666, 0.5, 1.0, 0.7142857142857143, 0.6153846153846154]
fmeasures=  [0.6386946386946387, 0.8678571428571429, 0.6715049656226126, 0.8181818181818181, 0.6985645933014354, 0.6476190476190476, 0.5561497326203209, 1.0, 0.7261904761904762, 0.6656611362493715]

In [20]:
import numpy as np
import scipy.stats as st

#define sample data
data = accs

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

data = precisions

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

data = recalls

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

data = fmeasures

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

(0.7061421141092344, 0.8547765627439455)
(0.818343629247924, 0.899097452700915)
(0.7061421194747853, 0.8547765651035212)
(0.716612808769828, 0.857250952913371)
